# Generate VWAP using dedicated service - Python

### Overview
The Volume Weighted Average price is similar to a moving average, except volume is included to weight the average price over a one trading day period.

It serves as a reference point for one trading day prices. Comparing current prices with the VWAP values enables to:
* mesure trading efficiency by comparing individual trades to VWAP
* highlight liquidity points as vwap reflects prices levels weighted by volume
* determine the intraday trend
* etc.

This sample demonstrates how to request and plot from a dedicated tick data service **on-demand** VWAP price.

### Inputs/outputs
VWAP indicator requires instrument's identifier, date time intervals and time range as per inputs and returns the VWAP data points.

VWAP is a cumulative indicator which means the number of data points progressively increases throughtout the day.

This sample shows how to plot a simple mid graph for basis technical analysis using an open source library. 

### Services used
This sample uses gRPC requests in order to retrieve VWAP tick data from the hosted service. The queried endpoint in this script are:
* *TickVwapService*: to directly retrieve vwap tick objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* and *plotly* as per display package

***

# Run VWAP sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly

In [ ]:
pip install -i https://test.pypi.org/simple/ systemathics.apis==0.9.27

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick_analytics.v1.tick_vwap_pb2 as tick_vwap
import systemathics.apis.services.tick_analytics.v1.tick_vwap_pb2_grpc as tick_vwap_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request
To request *tick VWAP* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick vwap request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2020, month = 2, day = 18), 
    end_date = date.Date(year = 2020, month = 2, day = 18)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 8, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 20, minutes = 0, seconds = 0)
)

#### 3.3 Request creation

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate the tick vwap request
request = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the tick vwap service
    service = tick_vwap_service.TickVwapServiceStub(channel)
    
    # process the tick vwap request
    time_stamps, trades, vwaps = [], [], []
    metadata = [('authorization', token)]
    for v in service.TickVwap(request=request, metadata=metadata):
        time_stamps.append(v.time_stamp)
        trades.append(v.trade)
        vwaps.append(v.vwap)

The following code snippet displays the total retrieved vwap data points count:

In [ ]:
print('Total top of the book limits retrieved: {0}'.format(len(vwaps)))

### Step 4: Visualize data

#### 4.1 Retrieve VWAP data
The following code snippets creates *gRPC client*, process request and returns the vwap request reply:

In [ ]:
# create the vwap dataframe
dates=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps]
d = {'Date': dates, 'Trade': trades, 'Vwap': vwaps}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot VWAP data with matplotlib

In [ ]:
import matplotlib.pyplot as plt

# create figure and axis objects with subplots()
fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'Vwap', data=df, marker='', color='red', linewidth=1, label="VWAP")
ax.plot('Date', 'Trade', data=df, marker='', color='blue', linewidth=1, label="Trades")
ax.set_ylabel("Trades and VWAP",color="black",fontsize=14)

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('Vwap and Trades for {0}-{1}'.format(ticker, exchange))
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# plot top of the book prices
fig = make_subplots()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Trade'], name='Trade', marker_color ='blue'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Vwap'], name='Vwap', marker_color ='red'))
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Tick vwap and trades for {0}-{1}'.format(ticker,exchange))
fig.show()